In [20]:
%matplotlib widget
import numpy as np
import pandas as pd
import random
import talib as ta
import math
from importlib import reload 
# from numba import jit

from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
vTest = 1000

''' 清空 stock_cal table '''
sql = "truncate table day_zhibiao"
basic.exec_sql(engine, sql)

data_dir = '../data/'  # for testing
# ts_list = pd.read_csv( '../data/sl_hs_const.csv')
# ts_list = pd.read_csv( data_dir + 'ts_all.csv')

# ts_list = pd.read_csv( data_dir + 'ts_mini.csv', dtype={'symbol':str, 'ts_code':str, 'name':str })

# 从 table 中读出来
# sql="SELECT * FROM sl_shengutong"
sql="SELECT * FROM stock_basic"
ts_list = pd.read_sql_query(sql, conn)
ts_list_size = len( ts_list )
ts_list.head(5), ts_list.shape 


(     ts_code  symbol  name area industry market list_date
 0  000001.SZ  000001  平安银行   深圳       银行     主板  19910403
 1  000002.SZ  000002   万科A   深圳     全国地产     主板  19910129
 2  000004.SZ  000004  国华网安   深圳     软件服务     主板  19910114
 3  000005.SZ  000005  ST星源   深圳     环境保护     主板  19901210
 4  000006.SZ  000006  深振业A   深圳     区域地产     主板  19920427,
 (5234, 7))

In [21]:
# import akshare as ak
# import func.base as bz 
# dfak = ak.stock_zh_a_spot_em()
# dfak.info()
# dfak.head(5)

In [22]:
# K线分析：引线+实体部分
def cal_upper(df):
    return ( df['high'] - max( df['open'], df['close'] ) ) / df['pre_close'] * 100
def cal_down(df):  
    return ( min( df['open'], df['close'] ) - df['low']  ) / df['pre_close'] * 100
def cal_range(df):  # 振幅%
    return ( df['high'] - df['low']  ) / df['pre_close'] * 100
def cal_st_high(df):  
    return df['open'] if df['open'] >= df['close'] else df['close']
def cal_st_low(df):
    return df['open'] if df['open'] < df['close'] else df['close']
def cal_st(df):  # 实体振幅
    return abs( df['open'] - df['close'] )
def cal_st_range(df):  # 实体振幅%
    return abs( df['open'] - df['close']  ) / df['pre_close'] * 100
# apply 进函数
def cal_pct_st():
    df['pct_upper'] = df.apply(cal_upper, axis=1)    # 上引线百分比
    df['pct_down']  = df.apply(cal_down,  axis=1)    # 下引线百分比
    df['pct_range'] = df.apply(cal_range, axis=1)    # 振幅百分比
    df['pct_st_range']  = df.apply(cal_st_range, axis=1)   # 实体部分百分比
    df['st_high']   = df.apply(cal_st_high,  axis=1)   # 实体高点价格
    df['st_low']    = df.apply(cal_st_low,   axis=1)   # 实体低点价格
    # df['pct_st']  = df.apply(cal_st, axis=1)   # 实体高度
    # 星线定义
    df['xing'] = df['pct_st_range']<=1
    return 'pct . '

# K线组合
# def k_red():

    # if df.iloc[index]['close'] >= df.iloc[i-1]['close'] and \

    # if df.iloc[index]['close'] >= df.iloc[i-1]['close'] and \
    #     # df.iloc[i-1]['close'] >= df.iloc[i-2]['close'] and \
    #     # df.iloc[i-2]['close'] >= df.iloc[i-3]['close'] :
    #     # if df.iloc[i-3]['close'] >= df.iloc[i-4]['close'] :        
    #     return 4
    # # else:
    # #     return 3
    # else:
    #     return 0 
    
    

In [23]:
# ma等
def cal_ma(mode):
    if mode=='ta':
        # yong ta-lib 包实现
        df.insert( df.shape[1], 'ma3', ta.MA(df['close'], 3) )
        df.insert( df.shape[1], 'ma5', ta.MA(df['close'], 5) )
        df.insert( df.shape[1], 'ma10', ta.MA(df['close'], 10) )
        df.insert( df.shape[1], 'ma20', ta.MA(df['close'], 20) )
        df.insert( df.shape[1], 'ma30', ta.MA(df['close'], 30) )
        df.insert( df.shape[1], 'ma60', ta.MA(df['close'], 60) )
        df.insert( df.shape[1], 'ma120', ta.MA(df['close'], 120) )
        df.insert( df.shape[1], 'ma250', ta.MA(df['close'], 250) )
        return 'ma-ta . '
    else:
        df['ma3']=df['close'].rolling(3).mean()
        df['ma5']=df['close'].rolling(5).mean()
        df['ma10']=df['close'].rolling(10).mean()
        df['ma20']=df['close'].rolling(20).mean()
        df['ma30']=df['close'].rolling(30).mean()
        df['ma60']=df['close'].rolling(60).mean()
        df['ma120']=df['close'].rolling(120).mean()
        df['ma250']=df['close'].rolling(250).mean()
        return 'ma-roll . '
        

In [24]:
''' 计算macd '''
def cal_macd():
    ''' way2 : ta-lib '''
    close = df.close.values  # 为 MACD 数据初始化close
    df['diff'], df['dea'], df['bar'] = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    df['bar'] = df['bar'] * 2
    return 'macd . '

# 输入df_name,算法参数。其中ewm函数算法快300-500倍
def cal_macd2(df, macd_way=2, ema_fast=12, ema_slow=26, dea_day=9):
    df['ema_fast'], df['ema_slow'] = 0.0, 0.0
    df['diff'], df['dea'], df['bar'] = 0.0,0.0,0.0

    df['ema_fast'] = df['close'].ewm(span=ema_fast,     adjust=False).mean()
    df['ema_slow'] = df['close'].ewm(span=ema_slow,     adjust=False).mean()
    df['diff'] = df['ema_fast'] - df['ema_slow']
    df['dea'] = df['diff'].ewm(span=9, adjust=False).mean()
    df['bar'] = (df['diff'] - df['dea']) * 2
    return 'macd2 . '

# def cal_more():
    # # 股价变化均值
    # df.insert( df.shape[1], 'ma3_pct_change', ta.MA(df['pct_change'], 3) )
    # # N天之内最大股价
    # df['MAX3_pct_change']=df['pct_change'].rolling(3).max()
    # # n天之内最高收盘价
    # df['MAX90_high']=df['high'].rolling(90).max()

# def cal_boll():
#     df['bb-u'], df['bb-m'], df['bb-l'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
#     df['upper'], df['middle'], df['lower'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )

In [25]:
""" 均线系列函数， """

# 计算在60线以下的days
def jx_ud60_count(val):
    if val <= 0:
        jx_ud60_count.count += 1
    else:
        jx_ud60_count.count = 0
    return jx_ud60_count.count

# 均线的斜率, 输出角度（不是度数）
def cal_jx_xl():
    df['jx_xl_5']  = (df['ma5']  - df['ma5'].rolling(2).mean()) * 2/  df['ma5']
    df['jx_xl_10'] = (df['ma10'] - df['ma10'].rolling(2).mean()) * 2/ df['ma10']
    df['jx_xl_20'] = (df['ma20'] - df['ma20'].rolling(2).mean()) * 2/ df['ma20']
    df['jx_xl_30'] = (df['ma30'] - df['ma30'].rolling(2).mean()) * 2/ df['ma30']
    df['jx_xl_60'] = (df['ma60'] - df['ma60'].rolling(2).mean()) * 2/ df['ma60']
    df['jx_xl_120'] = (df['ma120'] - df['ma120'].rolling(2).mean()) * 2/ df['ma120']
    df['jx_xl_250'] = (df['ma250'] - df['ma250'].rolling(2).mean()) * 2/ df['ma250']
    
    """ 字段优化处理 """
    # 斜率改为角度， 再乘100， 扩大影响
    df['jx_dg_5']  = df['jx_xl_5'].apply(np.arctan).apply(np.degrees)  
    df['jx_dg_10'] = df['jx_xl_10'].apply(np.arctan).apply(np.degrees) 
    df['jx_dg_20'] = df['jx_xl_20'].apply(np.arctan).apply(np.degrees) 
    df['jx_dg_30'] = df['jx_xl_30'].apply(np.arctan).apply(np.degrees) 
    df['jx_dg_60'] = df['jx_xl_60'].apply(np.arctan).apply(np.degrees) 
    df['jx_dg_120'] = df['jx_xl_120'].apply(np.arctan).apply(np.degrees) 
    df['jx_dg_250'] = df['jx_xl_250'].apply(np.arctan).apply(np.degrees)
    return  'xl . '

# 计算站上5日均线的天数
def jx_zs(val):
    if val >= 0:
        jx_zs.count += 1
    else:
        jx_zs.count = 0
    return jx_zs.count   

def cal_jx_zs_all():
    
    # jx - 站上5日均线天数
    df['zs_status'] = df['close'] - df['ma5']
    jx_zs.count = 0
    df['jx_zs_5'] = df['zs_status'].apply(jx_zs) #new column in dataframe
    # jx - 站上10日均线天数
    df['zs_status'] = df['close'] - df['ma10']
    jx_zs.count = 0
    df['jx_zs_10'] = df['zs_status'].apply(jx_zs)
    # jx - 站上20日均线天数
    df['zs_status'] = df['close'] - df['ma20']
    jx_zs.count = 0
    df['jx_zs_20'] = df['zs_status'].apply(jx_zs)
    # jx - 站上30日均线天数
    df['zs_status'] = df['close'] - df['ma30']
    jx_zs.count = 0
    df['jx_zs_30'] = df['zs_status'].apply(jx_zs)
    # print( end='30.' )
    
    # jx - 站上60日均线天数
    df['zs_status'] = df['close'] - df['ma60']
    jx_zs.count = 0
    df['jx_zs_60'] = df['zs_status'].apply(jx_zs)
    # print( end='60.' )
    # jx - 站上120日均线天数
    df['zs_status'] = df['close'] - df['ma120']
    jx_zs.count = 0
    df['jx_zs_120'] = df['zs_status'].apply(jx_zs)
    # print( end='120.' )
    # jx - 站上250日均线天数
    df['zs_status'] = df['close'] - df['ma250']
    jx_zs.count = 0
    df['jx_zs_250'] = df['zs_status'].apply(jx_zs)
    del df['zs_status']
    
    return('jx_zs 5-250. ')

In [26]:
""" 量价关系函数 """
# 成交量、x日均量
def cal_vol_ma():
    df['vol_ma3']=df['volume'].rolling(3).mean()
    df['vol_ma5']=df['volume'].rolling(5).mean()
    df['vol_ma10']=df['volume'].rolling(10).mean()
    df['vol_ma20']=df['volume'].rolling(20).mean()
    df['vol_ma30']=df['volume'].rolling(30).mean()
    df['vol_ma60']=df['volume'].rolling(60).mean()
    df['vol_ma120']=df['volume'].rolling(120).mean()
    df['vol_ma250']=df['volume'].rolling(250).mean()
    return 'vol_ma . '

# 成交量、x日均量
def cal_lj_fl():
    df['lj_fl_1_3']=df['volume'] / df['vol_ma3']
    df['lj_fl_3_10']=df['vol_ma3']/df['vol_ma10']
    df['lj_fl_5_20']=df['vol_ma5']/df['vol_ma20']
    df['lj_fl_5_60']=df['vol_ma5']/df['vol_ma60']
    df['lj_fl_20_60']=df['vol_ma20']/df['vol_ma60']
    # df['vol_ma60']=df['volume'].rolling(60).mean()
    # 放量正确化
    df['pct_fl_1_3']   = 2*df['lj_fl_1_3']   /(3  -   df['lj_fl_1_3'] )
    df['pct_fl_3_10']  = 7*df['lj_fl_3_10']  /(10 - 3*df['lj_fl_3_10'])
    df['pct_fl_5_20']  = 15*df['lj_fl_5_20'] /(20 - 5*df['lj_fl_5_20'])
    df['pct_fl_5_60']  = 55*df['lj_fl_5_60'] /(60 - 5*df['lj_fl_5_60'])
    df['pct_fl_20_60'] = 40*df['lj_fl_20_60']/(60 -20*df['lj_fl_20_60'])

    # 站上20日线
    # df['zs_20'] = (df['jx_zs_20'] > 0).astype(int)
    return 'pct_fl . '


In [27]:
ts_list

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,国华网安,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
5229,873339.BJ,873339,恒太照明,江苏,元器件,北交所,20221117
5230,873527.BJ,873527,夜光明,浙江,化工原料,北交所,20221027
5231,873576.BJ,873576,天力复合,陕西,小金属,北交所,20230712
5232,873593.BJ,873593,鼎智科技,江苏,机械基件,北交所,20230413


In [28]:
%%time

print('计算各类指标 : ')
for i in range( len(ts_list)  ):
# for i in range( 3  ):
    
    # begin with symbol
    print(ts_list.index[i]+1,' - ',str(ts_list_size), ts_list['symbol'][i], end=" ")
    
    # 不计算8字头，北京股票
    if ts_list['symbol'][i].startswith('8'):
        print( 'ignore 8xxx ...', end='\n' )
        continue
    # 不计算688字头，科创板
    if ts_list['symbol'][i].startswith('688'):
        print( 'ignore 688x ... ', end='\n' )
        continue
    
    # 本地MySQL读取
    sql = "select * from tran_daily where symbol='"+ts_list['symbol'][i]+"'"
    df = basic.read_data(conn, sql)
    df.set_index('trade_date', drop=True, append=False, inplace=True)
    df = df.sort_index(ascending=True)
    # 如果没有交易记录，则跳过loop
    if len(df) == 0:
        print( 'no data ... ', end='' )
        continue  
    
    # 纠错：xl_250等null数据填满、计算change、删除 
    # pre_close = 0
    df = df.fillna(value=0)
    df['change'] = df['close'] - df['pre_close']  
    df.drop(index=(df[ df['close']<=0 ].index),inplace=True)  # qfq模式下，股价负数的不再考虑。
    
    # K线分析：percentage 和 实体部分
    print( cal_pct_st(), end='' )
    
    # 计算K线组合：红三兵3
    df['close_f1']=df['close'].shift(1, axis=0, fill_value=0)
    df['close_f2']=df['close'].shift(2, axis=0, fill_value=0)
    df['close_f3']=df['close'].shift(3, axis=0, fill_value=0)
    # df['k_red'] = df[['close', 'close_f1']].apply(k_red) #new column in dataframe
    # df['k_red'] = (  df['df['close']>=df['close_f1'] and close_f1']>=df['close_f2'] ) 
    df['k_red'] = (df['close']>=df['close_f1']) & (df['close_f1']>=df['close_f2'])& (df['close_f2']>=df['close_f3'])  

    # df.drop(['C'       , 'D'      ], axis=1)
    df = df.drop(columns =['close_f1','close_f2','close_f3'])
    # df.drop(['close_f1','close_f2'], axis=1)
    # del  df['close_f1'] 
    # del  df['close_f2']
    # del  df['close_f3']
    
    print( 'k-red3, ', end='' )
    # 计算ma
    print( cal_ma('roll'), end='' )
    
    # 计算macd
    print( cal_macd(), end='' )
    # print( cal_macd2(df, 2, 12, 26, 9), end='' )
    # del df[['ema_fast','ema_slow']]
    # del df['ema_fast']
    # del df['ema_slow']

    # 计算junxian - 60线以下天数
    df['ud60_status'] = df['close'] - df['ma60']
    jx_ud60_count.count = 0
    df['jx_days_ud60'] = df['ud60_status'].apply(jx_ud60_count) #new column in dataframe
    del df['ud60_status']
    print( end='jx_ud60 . ' )

    # 计算cal_jx_xl - ma5,10,...250 均线斜率 
    print( cal_jx_xl(), end='' )
    # jx - 站上 x 日均线的天数
    print( cal_jx_zs_all(), end='' )

    """ 计算量价关系 """
    print( cal_vol_ma(), end='' )
    # def cal_vol_ma():
    # 计算放量情况
    print( cal_lj_fl(), end='' )
  
    """ 处理验证列， """
    # 后n天上涨率
    # if ts_list['symbol'][i] == '300474.SZ':  # for test purpose
    #     df['pct_change'] = [ random.randint(-2,2) for i in range(len(df)) ]
    df = df.sort_index(ascending=False)  # 倒着计算 sum 
    df['sz_sum_5'] = df['pct_change'].rolling(5+1).sum()  - df['pct_change']
    df['sz_sum_10'] = df['pct_change'].rolling(10+1).sum()  - df['pct_change']
    df['sz_sum_20'] = df['pct_change'].rolling(20+1).sum()  - df['pct_change']
    print( 'sz_sum. ', end='' )
    # 后n天最大上涨率    
    df['close_max'] = df['close'].rolling(5+1).max()
    df['sz_max_5'] = (df['close_max']-df['close'])*100/df['close']
    df['close_max'] = df['close'].rolling(10+1).max()
    df['sz_max_10'] = (df['close_max']-df['close'])*100/df['close']
    df['close_max'] = df['close'].rolling(20+1).max()
    df['sz_max_20'] = (df['close_max']-df['close'])*100/df['close']
    del df['close_max']
    print( 'sz_max. ', end='' )
    
    """ Final : 空值填充为0 """ 
    df = df.fillna(value=0)

    ''' 输出到 mysql '''
    # 删除当前symbol的已有指标
    # sql = "delete from day_zhibiao where symbol='"+ts_list['symbol'][i] +"'"
    # basic.exec_sql(conn, sql)
    write_no = df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000) 
    # index=True, 需要输出日期字段    

    print( end='\n')

print('\n\n指标运算结束')

计算各类指标 : 
1  -  5234 000001 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
2  -  5234 000002 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
3  -  5234 000004 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
4  -  5234 000005 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
5  -  5234 000006 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
6  -  5234 000007 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
7  -  5234 000008 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
8  -  5234 000009 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl . sz_sum. sz_max. 
9  -  5234 000010 pct . k-red3, ma-roll . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . pct_fl .

In [29]:
# df[df['close'] == 0]

In [30]:
# 数据测试 
sql = "select * from day_zhibiao where symbol='601318' order by trade_date desc "
# sql = "select * from day_zhibiao where symbol='000001' order by trade_date desc "
df = pd.read_sql_query(sql, conn, index_col=None)
# df

In [31]:
# columns = ['symbol','trade_date', 'open', 'close', 'high', 'low' ]
# columns = ['symbol','trade_date', 'ma5', 'ma10', 'ma20', 'ma30', 'ma60', 'ma120', 'ma250']
# columns = ['symbol','trade_date', 'jx_xl_5', 'jx_xl_10', 'jx_xl_20', 'jx_xl_30', 'jx_xl_60', 'jx_xl_120', 'jx_xl_250']
# columns = ['symbol','trade_date', 'jx_dg_5', 'jx_dg_10', 'jx_dg_20', 'jx_dg_30', 'jx_dg_60', 'jx_dg_120', 'jx_dg_250']
# columns = ['symbol','trade_date', 'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_5_20', 'pct_fl_20_60']
# columns = ['symbol','trade_date', 'open', 'close','volume', 'jx_zs_20']
columns = ['symbol','trade_date', 'pct_upper', 'pct_st_range', 'pct_down', 'xing']
df[columns].iloc[-200:, :]
# df[columns].iloc[0:100, :]


,symbol,trade_date,pct_upper,pct_st_range,pct_down,xing
1158,601318,2018-10-30,3.529410,5.400190,0.964320,0
1159,601318,2018-10-29,0.216646,6.986820,0.180538,0
1160,601318,2018-10-26,0.831711,1.344890,2.052730,0
1161,601318,2018-10-25,0.000000,4.385030,0.000000,0
1162,601318,2018-10-24,2.890170,0.848988,0.397399,1
...,...,...,...,...,...,...
1353,601318,2018-01-08,0.000000,1.323830,1.171080,0
1354,601318,2018-01-05,0.489782,1.182230,0.743118,0
1355,601318,2018-01-04,1.728810,0.271186,0.796610,1
1356,601318,2018-01-03,0.824810,4.008580,1.418670,0


In [32]:
# 调试：之后 n 天内的最大涨幅 -- reserved 

# df = df.sort_index(ascending=False)
# df['close_max'] = df['close'].rolling(5+1).max()
# df['sz_max_5'] = (df['close_max']-df['close'])/df['close']
# df.head(100)[['close', 'close_max', 'sz_max_5']]

In [33]:
# 调试：累计涨幅 -- reserved 
# if ts_list['symbol'] == '300474.SZ':
# df['pct_change'] = [ random.randint(-2,2) for i in range(len(df)) ]
# df = df.sort_index(ascending=False)
# df['pct_sum_5'] = df['pct_change'].rolling(5+1).sum()  - df['pct_change']
# df.head(15)[['pct_change', 'pct_sum_5']]

In [34]:
# received ...
# df['test'] = m_list = [random.randint(0,3) for i in range(1693)]
# df['test'] 
# df['pct_sum'] = df['test'].rolling(4).sum()  - df['test']
# df.head(12)[['test', 'pct_sum']]
# df = df.sort_index(ascending=False)



In [35]:
# df1 = df[ df['jx_days_ud60']==0 ]
# df1.head()
#筛选a列中值为a的行，形成新的df1
# df1 = df[ df['a'].isin(['a']) ]
#筛选列名为a、b、c的列，形成新的df1
# df1 = df[['a','b','c']]


In [36]:
# 插入和删除行，很有用
# df.insert( df.shape[1], 'ma250', ta.MA(df['close'], 250) )
# df.drop( labels=['MA125'], axis=1, inplace=True)
# df.head(3)